In [78]:
import numpy as np
import re
import pandas as pd
import tensorflow as tf

import os
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from tensorflow.contrib import learn


In [79]:

def clean_str(string):
  
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


In [80]:
class Text_CNN(object):
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        
        l2_loss = tf.constant(0.0)

        
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
        
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
        
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss


        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [81]:
def load_data_and_labels():
    df = pd.read_csv('spam.csv', encoding='latin-1')
    text = df.v2
    label = df.v1
    labels = []
    for _ in label:
        if _ == 'spam' :
            labels.append([1,0])
        else :
            labels.append([0,1])
    labels = np.array(labels)
    text = [clean_str(x) for x in text]
    
    return [text, labels]

In [82]:

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [83]:

def preprocess():
    
    x_text, y = load_data_and_labels()

    print(y[:5])
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    
    dev_sample_index = -1 * int(.1 * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev


In [84]:
def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=True,
          log_device_placement=False)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = Text_CNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=128,
                filter_sizes=list(map(int, [3,4,5])),
                num_filters=128,
                l2_reg_lambda=0.0)

            
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)

            
            vocab_processor.save(os.path.join(out_dir, "vocab"))

            
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
            
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 0.5
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
            
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            
            batches = batch_iter(
                list(zip(x_train, y_train)), 64, 50) #number of epochs 
            
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 100 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % 100 == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))


    

In [85]:

x_train, y_train, vocab_processor, x_dev, y_dev = preprocess()
train(x_train, y_train, vocab_processor, x_dev, y_dev)

[[0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]]
Vocabulary Size: 8705
Train/Dev split: 5015/557
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity i

2019-01-26T23:24:55.693972: step 96, loss 0.215805, acc 0.9375
2019-01-26T23:24:56.178082: step 97, loss 0.0373711, acc 0.984375
2019-01-26T23:24:56.660885: step 98, loss 0.177787, acc 0.9375
2019-01-26T23:24:57.140825: step 99, loss 0.0679193, acc 0.96875
2019-01-26T23:24:57.621962: step 100, loss 0.250917, acc 0.90625

Evaluation:
2019-01-26T23:24:59.012399: step 100, loss 0.0922412, acc 0.97307

Saved model checkpoint to /home/bardia/Documents/TA/CI/Project/runs/1548532450/checkpoints/model-100

2019-01-26T23:24:59.577050: step 101, loss 0.304323, acc 0.921875
2019-01-26T23:25:00.041567: step 102, loss 0.338927, acc 0.921875
2019-01-26T23:25:00.508816: step 103, loss 0.266717, acc 0.890625
2019-01-26T23:25:00.976647: step 104, loss 0.188882, acc 0.90625
2019-01-26T23:25:01.438758: step 105, loss 0.15191, acc 0.9375
2019-01-26T23:25:01.906476: step 106, loss 0.123264, acc 0.9375
2019-01-26T23:25:02.350126: step 107, loss 0.300438, acc 0.921875
2019-01-26T23:25:02.799122: step 108, lo

2019-01-26T23:25:48.332377: step 216, loss 0.140653, acc 0.953125
2019-01-26T23:25:48.736602: step 217, loss 0.0882356, acc 0.96875
2019-01-26T23:25:49.141230: step 218, loss 0.17978, acc 0.953125
2019-01-26T23:25:49.579787: step 219, loss 0.0486041, acc 0.96875
2019-01-26T23:25:49.995425: step 220, loss 0.0243765, acc 0.984375
2019-01-26T23:25:50.395978: step 221, loss 0.431712, acc 0.9375
2019-01-26T23:25:50.797666: step 222, loss 0.111726, acc 0.96875
2019-01-26T23:25:51.198124: step 223, loss 0.0574745, acc 0.96875
2019-01-26T23:25:51.621674: step 224, loss 0.0187981, acc 1
2019-01-26T23:25:52.020787: step 225, loss 0.131589, acc 0.9375
2019-01-26T23:25:52.419189: step 226, loss 0.0498169, acc 0.96875
2019-01-26T23:25:52.825890: step 227, loss 0.166486, acc 0.9375
2019-01-26T23:25:53.226893: step 228, loss 0.225352, acc 0.953125
2019-01-26T23:25:53.633595: step 229, loss 0.0612039, acc 0.96875
2019-01-26T23:25:54.026090: step 230, loss 0.141457, acc 0.984375
2019-01-26T23:25:54.430

2019-01-26T23:26:40.525307: step 340, loss 0.0145829, acc 1
2019-01-26T23:26:40.972733: step 341, loss 0.148165, acc 0.9375
2019-01-26T23:26:41.415706: step 342, loss 0.025517, acc 0.984375
2019-01-26T23:26:41.863339: step 343, loss 0.0533726, acc 0.96875
2019-01-26T23:26:42.308125: step 344, loss 0.000995091, acc 1
2019-01-26T23:26:42.767536: step 345, loss 0.0473097, acc 0.96875
2019-01-26T23:26:43.210691: step 346, loss 0.00182587, acc 1
2019-01-26T23:26:43.653393: step 347, loss 0.0360012, acc 0.96875
2019-01-26T23:26:44.096995: step 348, loss 0.0652739, acc 0.96875
2019-01-26T23:26:44.543475: step 349, loss 0.0100563, acc 1
2019-01-26T23:26:44.987888: step 350, loss 0.247142, acc 0.9375
2019-01-26T23:26:45.431883: step 351, loss 0.0150099, acc 1
2019-01-26T23:26:45.878965: step 352, loss 0.0408462, acc 0.984375
2019-01-26T23:26:46.325391: step 353, loss 0.0174304, acc 0.984375
2019-01-26T23:26:46.770005: step 354, loss 0.194338, acc 0.96875
2019-01-26T23:26:47.221109: step 355, lo

2019-01-26T23:27:36.326712: step 466, loss 0.148157, acc 0.96875
2019-01-26T23:27:36.791222: step 467, loss 0.0161797, acc 0.984375
2019-01-26T23:27:37.241242: step 468, loss 0.0639936, acc 0.96875
2019-01-26T23:27:37.689775: step 469, loss 0.0764795, acc 0.984375
2019-01-26T23:27:38.133934: step 470, loss 0.00156959, acc 1
2019-01-26T23:27:38.573885: step 471, loss 0.000756919, acc 1
2019-01-26T23:27:39.015378: step 472, loss 0.0107632, acc 1
2019-01-26T23:27:39.453442: step 473, loss 0.0438444, acc 0.984375
2019-01-26T23:27:39.649849: step 474, loss 0.103802, acc 0.956522
2019-01-26T23:27:40.097116: step 475, loss 0.000872729, acc 1
2019-01-26T23:27:40.526325: step 476, loss 0.0237954, acc 0.984375
2019-01-26T23:27:40.960597: step 477, loss 0.00101331, acc 1
2019-01-26T23:27:41.391259: step 478, loss 0.0812446, acc 0.984375
2019-01-26T23:27:41.826402: step 479, loss 0.0117772, acc 0.984375
2019-01-26T23:27:42.249906: step 480, loss 0.00577734, acc 1
2019-01-26T23:27:42.679588: step 4

...

2019-01-26T23:49:19.738776: step 3394, loss 0.000180156, acc 1
2019-01-26T23:49:20.175897: step 3395, loss 7.35738e-07, acc 1
2019-01-26T23:49:20.591404: step 3396, loss 1.08033e-07, acc 1
2019-01-26T23:49:20.766940: step 3397, loss 3.15397e-05, acc 1
2019-01-26T23:49:21.195097: step 3398, loss 5.96611e-05, acc 1
2019-01-26T23:49:21.612544: step 3399, loss 0.000420283, acc 1
2019-01-26T23:49:22.022711: step 3400, loss 5.68853e-05, acc 1

Evaluation:
2019-01-26T23:49:23.073248: step 3400, loss 0.0932973, acc 0.985637

Saved model checkpoint to /home/bardia/Documents/TA/CI/Project/runs/1548532450/checkpoints/model-3400

2019-01-26T23:49:23.566427: step 3401, loss 5.94305e-06, acc 1
2019-01-26T23:49:24.095132: step 3402, loss 4.92416e-05, acc 1
2019-01-26T23:49:24.662884: step 3403, loss 3.23324e-06, acc 1
2019-01-26T23:49:25.172552: step 3404, loss 2.68949e-06, acc 1
2019-01-26T23:49:25.697875: step 3405, loss 9.83305e-05, acc 1
2019-01-26T23:49:26.200135: step 3406, loss 1.20658e-05, ac

2019-01-26T23:50:19.991326: step 3519, loss 0.000655009, acc 1
2019-01-26T23:50:20.428594: step 3520, loss 0.002387, acc 1
2019-01-26T23:50:20.901241: step 3521, loss 2.60183e-05, acc 1
2019-01-26T23:50:21.338533: step 3522, loss 0.00216776, acc 1
2019-01-26T23:50:21.813196: step 3523, loss 0.000452787, acc 1
2019-01-26T23:50:22.270434: step 3524, loss 0.000153691, acc 1
2019-01-26T23:50:22.732781: step 3525, loss 2.78027e-05, acc 1
2019-01-26T23:50:23.172787: step 3526, loss 7.13897e-05, acc 1
2019-01-26T23:50:23.628126: step 3527, loss 0.000190027, acc 1
2019-01-26T23:50:24.054083: step 3528, loss 4.95823e-06, acc 1
2019-01-26T23:50:24.480520: step 3529, loss 1.42028e-05, acc 1
2019-01-26T23:50:24.907087: step 3530, loss 1.96318e-06, acc 1
2019-01-26T23:50:25.330613: step 3531, loss 7.29486e-05, acc 1
2019-01-26T23:50:25.772746: step 3532, loss 0.000127633, acc 1
2019-01-26T23:50:26.194008: step 3533, loss 7.54361e-05, acc 1
2019-01-26T23:50:26.767101: step 3534, loss 0.000258468, ac

2019-01-26T23:51:18.467939: step 3647, loss 1.28372e-05, acc 1
2019-01-26T23:51:18.916541: step 3648, loss 8.04657e-07, acc 1
2019-01-26T23:51:19.354626: step 3649, loss 1.39882e-06, acc 1
2019-01-26T23:51:19.801752: step 3650, loss 3.90688e-05, acc 1
2019-01-26T23:51:20.249532: step 3651, loss 1.61862e-06, acc 1
2019-01-26T23:51:20.697327: step 3652, loss 4.71198e-05, acc 1
2019-01-26T23:51:21.136265: step 3653, loss 5.90273e-05, acc 1
2019-01-26T23:51:21.589597: step 3654, loss 5.2469e-06, acc 1
2019-01-26T23:51:22.051574: step 3655, loss 8.82604e-05, acc 1
2019-01-26T23:51:22.493789: step 3656, loss 6.92899e-07, acc 1
2019-01-26T23:51:22.935325: step 3657, loss 0.000296211, acc 1
2019-01-26T23:51:23.366900: step 3658, loss 9.25466e-06, acc 1
2019-01-26T23:51:23.798794: step 3659, loss 8.23357e-06, acc 1
2019-01-26T23:51:24.222693: step 3660, loss 8.92202e-07, acc 1
2019-01-26T23:51:24.654022: step 3661, loss 1.98749e-05, acc 1
2019-01-26T23:51:25.082125: step 3662, loss 6.85393e-06,

2019-01-26T23:52:15.271982: step 3775, loss 5.64168e-06, acc 1
2019-01-26T23:52:15.683640: step 3776, loss 0.000263778, acc 1
2019-01-26T23:52:16.094626: step 3777, loss 0.00175834, acc 1
2019-01-26T23:52:16.508250: step 3778, loss 9.56955e-06, acc 1
2019-01-26T23:52:16.917338: step 3779, loss 0.000267973, acc 1
2019-01-26T23:52:17.322459: step 3780, loss 5.32691e-06, acc 1
2019-01-26T23:52:17.742032: step 3781, loss 0.000473084, acc 1
2019-01-26T23:52:18.155635: step 3782, loss 1.85494e-05, acc 1
2019-01-26T23:52:18.572166: step 3783, loss 2.09914e-06, acc 1
2019-01-26T23:52:18.984791: step 3784, loss 0.00264082, acc 1
2019-01-26T23:52:19.403655: step 3785, loss 8.26849e-06, acc 1
2019-01-26T23:52:19.817435: step 3786, loss 3.28209e-05, acc 1
2019-01-26T23:52:20.228874: step 3787, loss 8.15163e-05, acc 1
2019-01-26T23:52:20.640643: step 3788, loss 0.000741657, acc 1
2019-01-26T23:52:21.054939: step 3789, loss 6.5751e-07, acc 1
2019-01-26T23:52:21.470647: step 3790, loss 9.74753e-05, a

2019-01-26T23:53:12.107347: step 3901, loss 1.28892e-06, acc 1
2019-01-26T23:53:12.540817: step 3902, loss 4.89263e-06, acc 1
2019-01-26T23:53:12.990475: step 3903, loss 7.54491e-05, acc 1
2019-01-26T23:53:13.433059: step 3904, loss 5.24991e-05, acc 1
2019-01-26T23:53:13.868639: step 3905, loss 3.20464e-05, acc 1
2019-01-26T23:53:14.293768: step 3906, loss 2.47912e-06, acc 1
2019-01-26T23:53:14.726903: step 3907, loss 2.34869e-06, acc 1
2019-01-26T23:53:15.149054: step 3908, loss 6.38885e-07, acc 1
2019-01-26T23:53:15.579636: step 3909, loss 0.0207752, acc 0.984375
2019-01-26T23:53:16.006702: step 3910, loss 2.72272e-05, acc 1
2019-01-26T23:53:16.434125: step 3911, loss 0.000392408, acc 1
2019-01-26T23:53:16.866690: step 3912, loss 4.4217e-05, acc 1
2019-01-26T23:53:17.293739: step 3913, loss 2.52284e-05, acc 1
2019-01-26T23:53:17.729235: step 3914, loss 0.000819575, acc 1
2019-01-26T23:53:18.154519: step 3915, loss 9.01642e-06, acc 1
2019-01-26T23:53:18.590783: step 3916, loss 8.08313